# Windower Repo Example 02: Windower Pipeline Run

This notebook provides steps to preprocess the dataset (run Windower), use the preprocessed data to train the KitNet model, and run the mitigation simulation script. Analysis of the mitigation results and comparison to the original Kitsune are shown in the notebook `03_perf_comparison.ipynb`.

This notebook expects a PCAP dataset variant already prepared inside the `examples/work`. This can be achieved by running the `00_dataset.ipynb` notebook.

In [1]:
import os

In [2]:
# We expect a current directory to be /examples in order for these variables to work
SRC_DIR  = '../src'
WORK_DIR = 'work'

## Data Preprocessing

This section provides commands to perform feature extraction via Windower and prepare the data to feed the KitNet model.

In [3]:
# Label each packet as benign or malicious based on the provided attack_ips.txt file
!python $SRC_DIR/utils/pcap/dataset_label.py $WORK_DIR/ctu13_sc4_test.pcap $WORK_DIR/ctu13_sc4_test_attack_ips.txt $WORK_DIR/ctu13_sc4_test_pktlabels.txt

In [4]:
# See the Windower configuration
!cat $SRC_DIR/windower/config.yml | tail -n +13 | head -n -7

logger:
  # Length of the window in seconds
  window_length: 1

  # (Optional, default: 6) Minimum number of collected windows to process the given IP
  history_min: 6

  # (Optional, default: 0) Maximum number of elements that are stored in memory for
  # history. 0 refers to "infinity", allowing to store records to up 5GB of computer memory
  history_size: 0

  # (Optional, default: 120) Maximum number of seconds for which historical logs are valid
  history_timeout: 120

  # (Optional, default: 20) Minimum number of packets in the window to log it
  packets_min: 10

  # (Optional, default: 40) Number of samples for entropy estimation per IP per window
  samples_size: 40


In [5]:
%%time
# Prepare CTU-13 Scenario 4 train set
!python $SRC_DIR/windower/dataset_creator.py -c $SRC_DIR/windower/config.yml -n $WORK_DIR/ctu13_sc4_train.pcap $WORK_DIR/ctu13_sc4_train_extracols.csv

263131pkt [02:47, 1569.71pkt/s]
CPU times: user 3.47 s, sys: 572 ms, total: 4.04 s
Wall time: 2min 48s


In [6]:
%%time
# Make the dataset ready for Kitsune - drop columns that would cause artifacts and the file header
!python $SRC_DIR/utils/strip_cols.py $WORK_DIR/ctu13_sc4_train_extracols.csv $WORK_DIR/ctu13_sc4_train.csv keepfrag

CPU times: user 13.3 ms, sys: 11.6 ms, total: 24.9 ms
Wall time: 861 ms


## Model Training

In [7]:
# Determine the amount of samples within the training set
train_len = sum(1 for _ in open(os.path.join(WORK_DIR, 'ctu13_sc4_train.csv'))) - 1
train_len

1001

In [8]:
# Use 10% of samples for Kitsune scheme traning and the rest for training to AEs themselves
fmgrace = int(train_len * 0.1)
adgrace = train_len - fmgrace

In [9]:
%%time
# Perform Kitsune training
!python $SRC_DIR/kitsune/run-learning.py --csv -o $WORK_DIR/model_windower_kitnet.bin --fmgrace $fmgrace --adgrace $adgrace $WORK_DIR/ctu13_sc4_train.csv

INFO:utils:there are 1002 packets
INFO:KitNET.KitNET:Feature-Mapper: train-mode, Anomaly-Detector: off-mode
INFO:__main__:running learning
  0%|                                                  | 0/1002 [00:00<?, ?it/s]INFO:KitNET.KitNET:The Feature-Mapper found a mapping: 31 features to 11 autoencoders.
INFO:KitNET.KitNET:Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
100%|█████████████████████████████████████| 1002/1002 [00:00<00:00, 1061.12it/s]
INFO:__main__:learning finished
INFO:__main__:model written
CPU times: user 44.4 ms, sys: 10.1 ms, total: 54.5 ms
Wall time: 2.27 s


## Evaluation Running

In [10]:
# Show the configuration of the mitigation simulation
!cat $SRC_DIR/windower/config.yml | tail -n 6

mitig_simulator:
  # Threshold for marking a sample as anomalous
  threshold: 10

  # (Optional, default: 1000000) Size of the denylist (blacklist) in entries
  denylist_size: 1000000


As it might be seen, we have chosen the threshold of `10` to decide whether the reconstruction error of Kitnet (RMSE) is would be considered anomalous or not.

In [11]:
%%time
# Perform evaluation of the KitNet model trained with Windower statistics
!python $SRC_DIR/windower/mitig_simulator.py -c $SRC_DIR/windower/config.yml -a $WORK_DIR/ctu13_sc4_test_attack_ips.txt -m $WORK_DIR/model_windower_kitnet.bin -p $WORK_DIR/ctu13_sc4_test.pcap -d $WORK_DIR/predictions_windower.pkts -P $WORK_DIR/predictions_windower.wnds

929101pkt [11:05, 1395.94pkt/s]

------   Per-Source IP Communication Statistics   -----

               detected_after  detections_pos  detections_neg  pkts_allowed  pkts_denied   label
147.32.84.164               0               0              11          9619            0  Benign
147.32.80.9                 0               0            1557        328362            0  Benign
147.32.87.36                0               0              97        103263            0  Benign
147.32.84.170               0               0             417        163055            0  Benign
147.32.84.165               0               0              20          3633            0  Benign
147.32.87.11                0               0               0          1022            0  Benign
147.32.84.134               0               0             259        129288            0  Benign
10.0.0.165                166             189             125           275       190584  Attack

------   Model's Classification Stat

After this step, we used the trained model `model_windower_kitnet.bin` to simulate the DDoS mitigation process. The script provided a brief textual description of the mitigation process. Firstly, each IP address in the dataset is written to the output, and a series of statistics is provided. These statistics include:
- `detected_after` - Number of seconds the mechanism required to detect a malicious client
- `detections_pos` - Number of anomalous behavior detections for a given client
- `detections_neg` - Number of non-anomalous (supposed benign) behavior detections for a given client
- `pkts_allowed`   - Number of packets the mechanism let "pass-through" and reach its final destination
- `pkts_denied`    - Number of packets the mechanism marked as potentially malicious and decided to drop
- `label`          - True label of a client

In our simulation, we drop all packets after a particular client is marked anomalous. Despite dropping all its traffic, we still compute statistical windows and perform classifications, marking their results into `detections_pos` and `detections_neg` statistics. In practice, these values could be used to re-evaluate the status of maliciousness after a specific time has passed. In this example, we want to emphasize that even when an attacker acts non-anomalously in certain time windows (125 such classifications), we still block the majority of malicious traffic (0.999) as it was marked as anomalous several times before (189 events).

As the mitigation simulation shows, 1 of 1 attacker was detected successfully, whereas the mechanism blocked no legitimate clients. Due to more classifications and the attacker's apparent non-anomalous behavior patterns at certain times, the achieved recall is only `0.6` for window classification. Nevertheless, we block all malicious traffic after the first anomalous classification -- 99.9% of malicious traffic, while allowing 100% benign traffic.

In addition to the textual description, we obtained per-packet decisions `predictions_windower.pkts` file, which contains 0 or 1 values for each packet to determine whether the packet should be dropped or not, and `predictions_windower.wnds` containing RMSE scores of per-source IP windows produced by the Windower processed through Kitnet. These files will be used in the `03_perf_comparison.ipynb` notebook.